# Introduction to ffsim

ffsim is a software library for simulating fermionic quantum circuits that conserve particle number and the Z component of spin. By taking advantage of these symmetries, it can simulate these circuits much more efficiently than a generic quantum circuit simulator.

ffsim is a rather "low-level" software library. It does not have a class for representing quantum circuits. The primary way of using ffsim is by calling functions that transform statevectors represented directly as NumPy arrays. As an example, the following code shows how to create a Slater determinant and apply an orbital rotation to it.

In [1]:
import ffsim

# Set the number of orbitals and their occupancies
norb = 6
nelec = (3, 2)
n_alpha, n_beta = nelec
occupied_orbitals = (range(n_alpha), range(n_beta))

# Create a Slater determinant
vec = ffsim.slater_determinant(norb, occupied_orbitals=occupied_orbitals)

# Generate a random orbital rotation
orbital_rotation = ffsim.random.random_unitary(norb, seed=1234)

# Apply the orbital rotation to the statevector
rotated_vec = ffsim.apply_orbital_rotation(
    vec, orbital_rotation, norb=norb, nelec=nelec
)

Here, `vec` is a plain one-dimensional NumPy array (a vector). Its length is determined by the number of orbitals and their occupancies. Because ffsim only implements transformations which conserve particle number and the Z component of spin, the number of $\alpha$-electrons and the number of $\beta$-electrons are each fixed. For a system with $N$ spatial orbitals, $N_\alpha$ electrons with spin $\alpha$, and $N_\beta$ electrons with spin $\beta$, the statevector has length

$$
{N \choose N_\alpha} \times {N \choose N_\beta}.
$$

For convenience, ffsim includes functions to calculate these dimensions.

In [2]:
from scipy.special import comb

dim_a = comb(norb, n_alpha, exact=True)
dim_b = comb(norb, n_beta, exact=True)
dim = dim_a * dim_b
assert vec.shape == (dim,)
assert ffsim.dims(norb, nelec) == (dim_a, dim_b)
assert ffsim.dim(norb, nelec) == dim

This representation of the statevector is the same as that used in standard full configuration interaction (FCI) routines. It is often convenient to represent the statevector as a matrix whose rows are indexed by "$\alpha$-strings" describing the occupancies of the $\alpha$ orbitals, and columns indexed by "$\beta$-strings" describing the occupancies of the $\beta$ orbitals. To convert the vector into this representation, simply reshape it:

In [3]:
reshaped_vec = vec.reshape((dim_a, dim_b))

The statevector representation depends on a choice of ordering for the $\alpha$- and $\beta$-strings. ffsim uses the same ordering as pySCF's FCI module, `pyscf.fci`.